## Background and Research Question

### IIA

Independence of Irrelevant Alternatives (IIA) is a principle in decision theory that states that the preference between two options should not be affected by the presence or absence of a third option that is not relevant to the decision at hand.

In the context of recommendation systems, IIA means that the ranking of items should not change based on the addition or removal of items that are not relevant to the user's preferences. In other words, if a user is given a set of items to choose from and they express a preference for one item over another, the introduction of additional items that are not relevant to their preferences should not change the relative ranking of the original items.

Simple predictive models do not account for context effects, such as the dependence of choice behavior on the set of alternatives in a choice set. However, real world user choises are usually set dependent; violating IIA.

In this study, we aim to study the relation between the learned model and the obtained user choices in its domain. Specifically, we will compare user choices under IIA vs. non-IIA prediction models, and try to observe if the learned model "cause" user choices to be more IIA, or vice versa.



### Research Question

Do users make more IIA choices when using an IIA model compared to non-IIA model? If so, are user choices really set independent or is it a delusion caused by the IIA model?

### Hypothesis

- We expect to observe IIA user choices in IIA domains    
- More set dependent user choices in non IIA domains    
- We expect that IIA user behavior is a delusion made by the IIA model recommendations – e.g. extreme score for chosen items?

## Environment setup

In [1]:
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
# import abc
# import itertools
# from sklearn import svm, linear_model
# import pandas as pd
# import sys

from ContextChoiceEnv import *
from UserModel import *


from collections import OrderedDict

%config InlineBackend.figure_format = 'retina'

In [2]:
env = ContextChoiceEnvironment()
env.inspect_data("current")

,index,user_id,slate_id,x_0,x_1,E_s,rational_user_val,name
0,0,0.0,0.0,2.0,3.0,66.0,29.0,A
1,1,0.0,0.0,0.0,17.0,85.0,153.0,B
2,2,0.0,0.0,17.0,1.0,67.0,26.0,C
3,3,0.0,0.0,4.0,15.0,94.0,139.0,D
4,4,0.0,0.0,4.0,15.0,55.0,139.0,E
...,...,...,...,...,...,...,...,...
745,745,0.0,49.0,9.0,14.0,47.0,135.0,̪
746,746,0.0,49.0,12.0,6.0,67.0,66.0,̫
747,747,0.0,49.0,3.0,10.0,31.0,93.0,̬
748,748,0.0,49.0,6.0,10.0,97.0,96.0,̭


## Experiments

## Discussion